<a href="https://colab.research.google.com/github/mxpoch/PCF/blob/main/PCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# key words:
# savings = constant
# none = reserved for root parent

In [487]:
# authentication
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [488]:
# importing
import pandas as pd
import matplotlib.pyplot as plt
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1X4ahQA0pLWN3dd4X64pKShYLzDhvdM3frwBqAY7FfXY/edit?usp=sharing')

read_inputnodes = wb.worksheet('InputNodes').get_all_values()
read_indextree = wb.worksheet('IndexTree').get_all_values()

raw_inputnodes = pd.DataFrame(read_inputnodes[1:], columns=read_inputnodes[0])
# gspread imports everything as string, converting value and timeframe columns to numeric
raw_inputnodes = raw_inputnodes.drop(columns=['value', 'timeframe']).join(raw_inputnodes[['value', 'timeframe']].apply(pd.to_numeric))

raw_indextree = pd.DataFrame(read_indextree[1:], columns=read_indextree[0])

In [489]:
raw_inputnodes

,name,parent,value,timeframe
0,tuition,tuition&fees,-4000,120
1,sci_fees,tuition&fees,-8,30
2,food,root,-40,1
3,rent,money,-900,30


In [490]:
class IndexTree:
    def __init__(self, raw_indextree:pd.DataFrame(), raw_inputnodes:pd.DataFrame()): 
        # print(raw_indextree)
        self.indexdict = self.build_index_tree(raw_indextree, raw_inputnodes)
        self.valuedict = self.build_input_nodes(raw_inputnodes)

    def build_input_nodes(self, raw_inputnodes:pd.DataFrame()) -> dict:
        retdict = {}
        for i in range(raw_inputnodes.shape[0]):
            current = raw_inputnodes.iloc[i]
            retdict[current['name'].replace(' ', '')] = {col:current[col] for col in current.index if col != 'name'}
        return retdict

    def build_index_tree(self, raw_indextree:pd.DataFrame(), raw_inputnodes:pd.DataFrame()) -> dict:
        retdict = {}
        for i in range(raw_indextree.shape[0]):
            current = raw_indextree.iloc[i]
            children = raw_indextree[raw_indextree['parent'] == current['name']]
            node_children = raw_inputnodes[raw_inputnodes['parent'] == current['name']]
            
            if children.shape[0] == 0 and node_children.shape[0] == 0:
                retdict[current['name']] = 0
                continue

            children = children['name'].tolist()+node_children['name'].tolist()
            retdict[current['name']] = list(map(lambda x: x.replace(' ', ""), children)) #cleans up any mistaken spaces
        
        return retdict
    
    def sum_all_worker(self, name:str, currentsum:list, timeframe:int):
        if name not in self.indexdict.keys():
            currentsum[0] += self.get_current_value(name, timeframe)
            return

        for next in self.indexdict[name]:
            self.sum_all_worker(next, currentsum, timeframe)
    
    def sum_all(self, name:str, timeframe:int) -> int:
        currentsum = [0] # necessary to pass by reference in python
        self.sum_all_worker(name, currentsum, timeframe)
        return float(currentsum[0])

    def get_current_value(self, name:str, timeframe:int) -> int:
        return self.valuedict[name]['value']*(timeframe/self.valuedict[name]['timeframe'])

    def pretty_worker(self, name:str, recurdepth:int):
        print("   "*recurdepth, name,"$"+str(self.sum_all(name, 30)))
        if name not in self.indexdict.keys():
            return
        for next in self.indexdict[name]:
            self.pretty_worker(next, recurdepth+1)
    
    def pretty_print_index(self):
        print('Nodes:')
        self.pretty_worker('root', 0) 

In [491]:
IT = IndexTree(raw_indextree, raw_inputnodes)

In [492]:
IT.sum_all('tuition&fees', 120)

-4032.0

In [493]:
IT.pretty_print_index()

Nodes:
 root $-3108.0
    money $-1908.0
       tuition&fees $-1008.0
          tuition $-1000.0
          sci_fees $-8.0
       rent $-900.0
    food $-1200.0
